In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader

In [10]:
# 데이터 전처리 및 데이터셋 로드
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지 정규화
])

# CIFAR-100 데이터셋 로드 (훈련 데이터 및 테스트 데이터)
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

# CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 입력 채널 3 (RGB), 출력 채널 32
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 출력 채널 64
        self.pool = nn.MaxPool2d(2, 2)  # 풀링 계층
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # 평탄화 후 입력 크기
        self.fc2 = nn.Linear(512, 100)  # CIFAR-100은 100개의 클래스를 가짐

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # 64개의 8x8 크기 feature map을 평탄화
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 인스턴스화
model = SimpleCNN()

# 손실 함수 및 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 훈련 함수
def train(model, trainloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()  # 기울기 초기화
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 기울기 계산
            optimizer.step()  # 매개변수 업데이트
            running_loss += loss.item()
            if i % 100 == 99:  # 100 미니배치마다 출력
                print(f"Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}")
                running_loss = 0.0

# 테스트 함수
def test(model, testloader):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    with torch.no_grad():  # 그래디언트 계산 안 함
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # 예측값 선택
            total += labels.size(0)  # 전체 데이터 수
            correct += (predicted == labels).sum().item()  # 정확도 계산

    print(f'Accuracy on test set: {100 * correct / total}%')

# 모델 훈련 실행
train(model, trainloader, criterion, optimizer, num_epochs=10)

# 테스트 실행
test(model, testloader)

Epoch 1, Batch 100, Loss: 4.587
Epoch 1, Batch 200, Loss: 4.394
Epoch 1, Batch 300, Loss: 4.125
Epoch 1, Batch 400, Loss: 4.014
Epoch 1, Batch 500, Loss: 3.903
Epoch 1, Batch 600, Loss: 3.735
Epoch 1, Batch 700, Loss: 3.689
Epoch 1, Batch 800, Loss: 3.553
Epoch 1, Batch 900, Loss: 3.514
Epoch 1, Batch 1000, Loss: 3.443
Epoch 1, Batch 1100, Loss: 3.354
Epoch 1, Batch 1200, Loss: 3.382
Epoch 1, Batch 1300, Loss: 3.248
Epoch 1, Batch 1400, Loss: 3.267
Epoch 1, Batch 1500, Loss: 3.176
Epoch 2, Batch 100, Loss: 3.014
Epoch 2, Batch 200, Loss: 3.014
Epoch 2, Batch 300, Loss: 2.978
Epoch 2, Batch 400, Loss: 2.965
Epoch 2, Batch 500, Loss: 2.876
Epoch 2, Batch 600, Loss: 2.866
Epoch 2, Batch 700, Loss: 2.844
Epoch 2, Batch 800, Loss: 2.793
Epoch 2, Batch 900, Loss: 2.816
Epoch 2, Batch 1000, Loss: 2.773
Epoch 2, Batch 1100, Loss: 2.732
Epoch 2, Batch 1200, Loss: 2.738
Epoch 2, Batch 1300, Loss: 2.691
Epoch 2, Batch 1400, Loss: 2.624
Epoch 2, Batch 1500, Loss: 2.685
Epoch 3, Batch 100, Loss: 2.